## Setup for Magic Eraser Models
---
In this notebook, you are going to run a defined setup process for our magic eraser solution. Due to the size of the models, some of the step may take some time to complete. The entire notebook should finish within 1 hour. At the end of the notebook run, we should have a instended inference container in Elastic Container Registry (ECR) ready to host our models using SageMaker Endpoint (MME).

SageMaker MME is a service provided by Amazon SageMaker that allows multiple machine learning models to be hosted on a single endpoint. This means that multiple models can be deployed and managed together, making it easier to scale and maintain machine learning applications. With a multi-model endpoint, different models can be selected based on specific needs, allowing for more flexibility and efficiency. It also enables different types of models to be combined, such as computer vision and natural language processing models, to create more comprehensive applications.

Here is a high level breakdown of the setup steps:

1. Downloading pre-trained models
2. Package conda environment for additional model dependencies
3. Extend SageMaker managed Triton container with model checkpoints and conda packs pre-loaded
4. Push the container to AWS Elastic Container Registry (ECR)

---

This notebook will locally build a custom docker image. **It is recommended to run this example outside of SageMaker Studio**

In [11]:
!pip install -Uq nvidia-pyindex 
!pip install -Uq sagemaker
!pip install -Uq gradio

### Setup

In [12]:
import sagemaker
import boto3

import tarfile
import os

sagemaker_session = sagemaker.Session(boto_session=boto3.Session())
region = sagemaker_session.boto_region_name
account = sagemaker_session.account_id()

### Download Pre-trained Models

#### Download and Package SAM Checkpoint
**Apache-2.0 license**

In [3]:
model_file_name = "sam_vit_h_4b8939.pth"
download_path = f"https://huggingface.co/spaces/abhishek/StableSAM/resolve/main/{model_file_name}"

!wget $download_path

--2023-05-15 21:26:35--  https://huggingface.co/spaces/abhishek/StableSAM/resolve/main/sam_vit_h_4b8939.pth
Resolving huggingface.co (huggingface.co)... 18.161.6.70, 18.161.6.77, 18.161.6.48, ...
Connecting to huggingface.co (huggingface.co)|18.161.6.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/47/d3/47d331d77ce5639cc128df17410f4744b11342191e8442f5cde65f20735d01f9/a7bf3b02f3ebf1267aba913ff637d9a2d5c33d3173bb679e46d9f338c26f262e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sam_vit_h_4b8939.pth%3B+filename%3D%22sam_vit_h_4b8939.pth%22%3B&Expires=1684445196&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQ3L2QzLzQ3ZDMzMWQ3N2NlNTYzOWNjMTI4ZGYxNzQxMGY0NzQ0YjExMzQyMTkxZTg0NDJmNWNkZTY1ZjIwNzM1ZDAxZjkvYTdiZjNiMDJmM2ViZjEyNjdhYmE5MTNmZjYzN2Q5YTJkNWMzM2QzMTczYmI2NzllNDZkOWYzMzhjMjZmMjYyZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc

In [4]:
sd_tar = f"docker/{model_file_name}.tar.gz"

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

make_tarfile(sd_tar, model_file_name)

### Download and Package LaMa Checkpoint
**Apache-2.0 license**

In [5]:
!wget https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip
!unzip big-lama.zip

--2023-05-15 21:32:38--  https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip
Resolving huggingface.co (huggingface.co)... 18.161.6.70, 18.161.6.27, 18.161.6.77, ...
Connecting to huggingface.co (huggingface.co)|18.161.6.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/51/45/51456bfbae988f737d159c165ee79a3168fdd10d70db51da3e2dcc0cb29aa7a5/f1b358ca24093b93a106183b98a3dea6e8ed09f3b43ea7251eb2c81e7b4575f6?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27big-lama.zip%3B+filename%3D%22big-lama.zip%22%3B&response-content-type=application%2Fzip&Expires=1684445559&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzUxLzQ1LzUxNDU2YmZiYWU5ODhmNzM3ZDE1OWMxNjVlZTc5YTMxNjhmZGQxMGQ3MGRiNTFkYTNlMmRjYzBjYjI5YWE3YTUvZjFiMzU4Y2EyNDA5M2I5M2ExMDYxODNiOThhM2RlYTZlOGVkMDlmM2I0M2VhNzI1MWViMmM4MWU3YjQ1NzVmNj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVud

In [6]:
lama_dir = "big-lama"
lama_tar = f"docker/{lama_dir}.tar.gz"

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

make_tarfile(lama_tar, lama_dir)

LaMa needs additional source code script. We will clone the repo into our `model_repo` folder

In [10]:
!cd model_repo/lama/1 && git clone https://github.com/advimman/lama.git

Cloning into 'lama'...
remote: Enumerating objects: 399, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 399 (delta 76), reused 68 (delta 55), pack-reused 271
Receiving objects: 100% (399/399), 9.87 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (138/138), done.


#### Downloading Images and Modules from Inpaint Anything
**Apache-2.0 license**

In [13]:
!cd statics && wget https://github.com/geekyutao/Inpaint-Anything/blob/main/example/fill-anything/sample1.png
!cd statics && wget https://github.com/geekyutao/Inpaint-Anything/blob/main/example/remove-anything/dog.jpg

--2023-05-20 20:22:31--  https://github.com/geekyutao/Inpaint-Anything/blob/main/example/fill-anything/sample1.png
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘sample1.png’

    [ <=>                                   ] 143,273     --.-K/s   in 0.03s   

2023-05-20 20:22:32 (4.20 MB/s) - ‘sample1.png’ saved [143273]

--2023-05-20 20:22:32--  https://github.com/geekyutao/Inpaint-Anything/blob/main/example/remove-anything/dog.jpg
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘dog.jpg’

    [ <=>                                   ] 143,197     --.-K/s   in 0.03s   

2023-05-20 20:22:32 (5.18 MB/s) - ‘dog.jpg’ saved [143197]



### Package Conda Environment for each model

SageMaker NVIDIA Triton container images does not contain all the libraries two run our SAM and LaMa models. However, Triton allows you to bring additional dependencies using conda pack. Let's run the two cells below to create a `xxx_env.tar.gz` environment package for each model.

In [7]:
!cd docker && bash sam_conda_dependencies.sh 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/sam_env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.5.7   |       hbcca054_0         145 KB  conda-forge
    libsqlite-3.41.2           |       h2797004_1         800 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         945 KB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge None
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu None
  bzip2              conda-forge

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 394.7 MB/s eta 0:00:00
  Created wheel for lit: filename=lit-16.0.3-py3-none-any.whl size=88173 sha256=0b1a8d36deab986f4da212c45707df8a18bdf389d62458308dc8cf0a8490b7c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-jsxmidzv/wheels/01/20/c2/b32a74e477abe541734225f8d5ed8b9e284d1f8580f6828445
Successfully built lit
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 280.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 284.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 289.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 335.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.9/1

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 124.7 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 139.3 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 374.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 402.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 404.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 274.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 317.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 399.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com

In [8]:
!cd docker && bash lama_conda_dependencies.sh

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/lama_env

  added / updated specs:
    - python=3.8


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge None
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu None
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h7f98852_4 None
  ca-certificates    conda-forge/linux-64::ca-certificates-2023.5.7-hbcca054_0 None
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-h41732ed_0 None
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 None
  libgcc-ng          conda-forge/linux-64::libgcc-ng-12.2.0-h65d4601_19 None
  libgomp            conda-forge/linux-64::libgomp-12.2.0-h65d4601_19 None
  

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 276.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 267.4 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 kB 18.2 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
  Created wheel for easydict: filename=easyd

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 265.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 387.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 247.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 371.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 305.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 316.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 180.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 407.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 253.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 362.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 389.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 348.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.9/195.9 kB 315.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 309.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 354.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 337.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 360.7 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.10-py3-none-any.whl size=1480503 sha256=51164eb7f41b237a46f7c49f68aaaf003e194313c9e5f20c25a174f6fde05971
  Stored in directory: /tmp/pip-ephem-wheel-cache-bv89fec2/wheels/6a/8c/35/209bfba5979c08e883a5cfcb9f3ddf907f9dd1c37ee83c1f8d

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 279.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 330.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 309.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 300.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141210 sha256=b7ba56330d47319dbbb5d157f09f72cd7e2bb62137e

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 378.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 330.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 342.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 323.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 348.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 339.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 kB 364.1 MB/s eta 0:00:00


  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492022 sha256=a021e5eb1c9350f151503dccbb5e2b4bbefd5eebd29150ebdd18a3cc7a1b575e
  Stored in directory: /tmp/pip-ephem-wheel-cache-g24z6rxc/wheels/a0/0b/ee/e6994fadb42c1354dcccb139b0bf2795271bddfe6253ccdf11
Successfully built future
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 kB 8.8 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 317.0 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for conda-pack: filename=conda_pack-0.6.0-py2.py3-none-any.whl size=30883 sha256=e1ae3a87cdf912676ba8f15803a45fba12f56d3a44d41a9

[########################################] | 100% Completed |  3min 42.8s


### Extend SageMaker Managed Triton Container

When we host these models on SageMaker MME. When invoked, model files will be loaded from S3 onto the instance. Due the large size of our models and model packages (SAM: 2.4GB + conda pack: 2.52 GB, LaMa: 0.38 GB + conda pack: 3.35GB), we are going to pre-load these files into the container. This will reduce model loading time and improve user experience during cold start.

In [2]:
# account mapping for SageMaker Triton Image
account_id_map = {
    "us-east-1": "785573368785",
    "us-east-2": "007439368137",
    "us-west-1": "710691900526",
    "us-west-2": "301217895009",
    "eu-west-1": "802834080501",
    "eu-west-2": "205493899709",
    "eu-west-3": "254080097072",
    "eu-north-1": "601324751636",
    "eu-south-1": "966458181534",
    "eu-central-1": "746233611703",
    "ap-east-1": "110948597952",
    "ap-south-1": "763008648453",
    "ap-northeast-1": "941853720454",
    "ap-northeast-2": "151534178276",
    "ap-southeast-1": "324986816169",
    "ap-southeast-2": "355873309152",
    "cn-northwest-1": "474822919863",
    "cn-north-1": "472730292857",
    "sa-east-1": "756306329178",
    "ca-central-1": "464438896020",
    "me-south-1": "836785723513",
    "af-south-1": "774647643957",
}



region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise ("UNSUPPORTED REGION")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.12-py3".format(
        account_id=account_id_map[region], region=region, base=base
    )
)
triton_account_id = account_id_map[region]
mme_triton_image_uri

'301217895009.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tritonserver:22.12-py3'

Preview the docker file

In [3]:
!cat docker/Dockerfile

ARG BASE_IMAGE

FROM $BASE_IMAGE

#Install any additional libraries
RUN echo "Adding conda package to Docker image"
RUN mkdir -p /home/condpackenv/
RUN mkdir -p /home/models/

# Copy conda env
COPY sam_env.tar.gz /home/condpackenv/sam_env.tar.gz
COPY lama_env.tar.gz /home/condpackenv/lama_env.tar.gz

COPY sam_vit_h_4b8939.pth.tar.gz /temp/
COPY big-lama.tar.gz /temp/

# Install tar
RUN apt-get update && apt-get install -y tar
RUN apt-get update && apt-get install ffmpeg libsm6 libxext6  -y

# Untar the file
RUN tar -xzf /temp/sam_vit_h_4b8939.pth.tar.gz -C /home/models/
RUN tar -xzf /temp/big-lama.tar.gz -C /home/models/

RUN rm /temp/sam_vit_h_4b8939.pth.tar.gz
RUN rm /temp/big-lama.tar.gz

### Build & push the new image to ECR

In [4]:
# New container image name
new_image_name = 'sagemaker-tritonserver-sam-lama'

In [5]:
%%capture build_output
!cd docker && bash build_and_push.sh "$new_image_name" "latest" "$mme_triton_image_uri" "$region" "$account" "$triton_account_id"

In [6]:
if 'Error response from daemon' in str(build_output):
    print(build_output)
    raise SystemExit('\n\n!!There was an error with the container build!!')
else:
    extended_triton_image_uri = str(build_output).strip().split('\n')[-1]
    
print(f"New image URI from ECR: {extended_triton_image_uri}")

New image URI from ECR: 376678947624.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tritonserver-sam-lama:latest


In [7]:
%store extended_triton_image_uri

Stored 'extended_triton_image_uri' (str)
